In [1]:
import numpy as np
import pandas as pd
from collections import Counter, defaultdict
from dataclasses import dataclass, field
from typing import List, Dict, Tuple, Optional
from itertools import combinations
from abc import ABC, abstractmethod
from datetime import datetime
import json
import warnings
warnings.filterwarnings('ignore')

# ============================================================
#                    CONFIGURATION
# ============================================================

@dataclass
class GameConfig:
    name: str
    num_max: int
    nums_per_draw: int
    filepath: str

CONFIGS = {
    "loto": GameConfig("Loto", 49, 5, "lotodata.csv"),
    "keno": GameConfig("Keno", 70, 20, "kenodata.csv"),
    "euromillions": GameConfig("EuroMillions", 50, 5, "euromillions.csv"),
}

# ============================================================
#              FONCTIONS UTILITAIRES
# ============================================================

def sigmoid(x):
    x = np.clip(x, -500, 500)
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# ============================================================
#         SYSTÈME DE SUIVI DES PERFORMANCES
# ============================================================

@dataclass
class PredictionRecord:
    """Enregistrement d'une prédiction."""
    date: str
    draw_index: int
    predicted_numbers: List[int]
    actual_numbers: List[int]
    module_name: str
    hits: int = 0
    
    def __post_init__(self):
        self.hits = len(set(self.predicted_numbers) & set(self.actual_numbers))
    
    @property
    def hit_rate(self) -> float:
        return self.hits / len(self.predicted_numbers) if self.predicted_numbers else 0


class PerformanceTracker:
    """
    Suivi et analyse des performances des prédictions.
    Stocke l'historique et calcule les métriques.
    """
    
    def __init__(self, config: GameConfig):
        self.config = config
        self.records: List[PredictionRecord] = []
        self.module_stats: Dict[str, Dict] = defaultdict(lambda: {
            'total_predictions': 0,
            'total_hits': 0,
            'hit_rates': [],
            'best_prediction': 0,
            'worst_prediction': float('inf')
        })
        
    def add_record(self, record: PredictionRecord):
        """Ajoute un enregistrement de prédiction."""
        self.records.append(record)
        
        # Mettre à jour les stats du module
        stats = self.module_stats[record.module_name]
        stats['total_predictions'] += 1
        stats['total_hits'] += record.hits
        stats['hit_rates'].append(record.hit_rate)
        stats['best_prediction'] = max(stats['best_prediction'], record.hits)
        stats['worst_prediction'] = min(stats['worst_prediction'], record.hits)
        
    def get_module_performance(self, module_name: str) -> Dict:
        """Retourne les statistiques de performance d'un module."""
        stats = self.module_stats[module_name]
        if stats['total_predictions'] == 0:
            return {'avg_hit_rate': 0, 'total_hits': 0}
        
        return {
            'total_predictions': stats['total_predictions'],
            'total_hits': stats['total_hits'],
            'avg_hit_rate': np.mean(stats['hit_rates']),
            'std_hit_rate': np.std(stats['hit_rates']),
            'best_prediction': stats['best_prediction'],
            'worst_prediction': stats['worst_prediction'],
            'consistency_score': 1 - np.std(stats['hit_rates']) if stats['hit_rates'] else 0
        }
    
    def get_overall_performance(self) -> Dict:
        """Retourne les performances globales."""
        if not self.records:
            return {}
        
        all_hits = [r.hits for r in self.records]
        all_rates = [r.hit_rate for r in self.records]
        
        return {
            'total_records': len(self.records),
            'avg_hits': np.mean(all_hits),
            'avg_hit_rate': np.mean(all_rates),
            'best_ever': max(all_hits),
            'total_correct': sum(all_hits)
        }
    
    def get_number_accuracy(self) -> Dict[int, float]:
        """Calcule la précision de prédiction pour chaque numéro."""
        predicted_counts = Counter()
        hit_counts = Counter()
        
        for record in self.records:
            for num in record.predicted_numbers:
                predicted_counts[num] += 1
                if num in record.actual_numbers:
                    hit_counts[num] += 1
        
        accuracy = {}
        for num in range(1, self.config.num_max + 1):
            if predicted_counts[num] > 0:
                accuracy[num] = hit_counts[num] / predicted_counts[num]
            else:
                accuracy[num] = 0.0
        
        return accuracy
    
    def save_to_file(self, filename: str = "prediction_history.json"):
        """Sauvegarde l'historique des prédictions."""
        data = {
            'config': {
                'name': self.config.name,
                'num_max': self.config.num_max,
                'nums_per_draw': self.config.nums_per_draw
            },
            'records': [
                {
                    'date': r.date,
                    'draw_index': r.draw_index,
                    'predicted': r.predicted_numbers,
                    'actual': r.actual_numbers,
                    'module': r.module_name,
                    'hits': r.hits
                }
                for r in self.records
            ]
        }
        with open(filename, 'w') as f:
            json.dump(data, f, indent=2)
        print(f"✅ Historique sauvegardé dans {filename}")
    
    def load_from_file(self, filename: str = "prediction_history.json"):
        """Charge l'historique des prédictions."""
        try:
            with open(filename, 'r') as f:
                data = json.load(f)
            
            for r in data['records']:
                record = PredictionRecord(
                    date=r['date'],
                    draw_index=r['draw_index'],
                    predicted_numbers=r['predicted'],
                    actual_numbers=r['actual'],
                    module_name=r['module']
                )
                self.add_record(record)
            
            print(f"✅ {len(self.records)} enregistrements chargés")
        except FileNotFoundError:
            print(f"⚠️  Fichier {filename} non trouvé")


# ============================================================
#              MOTEUR DE BACKTESTING
# ============================================================

class BacktestEngine:
    """
    Moteur de backtesting pour évaluer les stratégies sur l'historique.
    Simule les prédictions passées et compare aux résultats réels.
    """
    
    def __init__(self, config: GameConfig, historique: List[List[int]]):
        self.config = config
        self.historique = historique
        self.results: Dict[str, List[Dict]] = defaultdict(list)
        
    def run_backtest(self, module, 
                     train_size: int = 100,
                     test_size: int = 50,
                     step: int = 1,
                     top_n: int = None) -> Dict:
        """
        Exécute un backtest sur un module.
        
        Args:
            module: Le module de prédiction à tester
            train_size: Nombre de tirages pour l'entraînement
            test_size: Nombre de tirages à prédire
            step: Pas entre chaque test
            top_n: Nombre de numéros à prédire (défaut: nums_per_draw)
        """
        if top_n is None:
            top_n = self.config.nums_per_draw
            
        if len(self.historique) < train_size + test_size:
            print(f"⚠️  Pas assez de données pour le backtest")
            return {}
        
        predictions = []
        actuals = []
        hits_list = []
        
        # Parcourir l'historique avec une fenêtre glissante
        for i in range(train_size, len(self.historique) - 1, step):
            if i + 1 >= len(self.historique):
                break
                
            # Entraîner sur les données passées
            module.historique = self.historique[:i]
            module.compute_scores()
            
            # Prédire les top_n numéros
            predicted = module.get_top_numbers(top_n)
            
            # Comparer avec le tirage réel suivant
            actual = self.historique[i]
            
            hits = len(set(predicted) & set(actual))
            
            predictions.append(predicted)
            actuals.append(actual)
            hits_list.append(hits)
            
            self.results[module.name].append({
                'train_end': i,
                'predicted': predicted,
                'actual': actual,
                'hits': hits,
                'hit_rate': hits / top_n
            })
        
        # Calculer les statistiques
        if not hits_list:
            return {}
            
        return {
            'module_name': module.name,
            'total_tests': len(hits_list),
            'avg_hits': np.mean(hits_list),
            'std_hits': np.std(hits_list),
            'max_hits': max(hits_list),
            'min_hits': min(hits_list),
            'avg_hit_rate': np.mean([h/top_n for h in hits_list]),
            'hit_distribution': dict(Counter(hits_list)),
            'predictions_sample': predictions[-5:] if predictions else []
        }
    
    def compare_modules(self, modules: List, **kwargs) -> pd.DataFrame:
        """Compare les performances de plusieurs modules."""
        results = []
        
        print("\n🔬 Backtesting en cours...")
        for module in modules:
            print(f"   Testing {module.name}...", end=" ")
            stats = self.run_backtest(module, **kwargs)
            if stats:
                results.append(stats)
                print(f"✓ (avg hits: {stats['avg_hits']:.2f})")
            else:
                print("✗")
        
        if not results:
            return pd.DataFrame()
            
        df = pd.DataFrame(results)
        df = df.sort_values('avg_hits', ascending=False)
        return df
    
    def get_optimal_weights(self) -> Dict[str, float]:
        """
        Calcule les poids optimaux pour chaque module basé sur leurs performances.
        """
        weights = {}
        total_performance = 0
        
        for module_name, results in self.results.items():
            if results:
                avg_hits = np.mean([r['hits'] for r in results])
                consistency = 1 / (np.std([r['hits'] for r in results]) + 0.1)
                performance = avg_hits * consistency
                weights[module_name] = performance
                total_performance += performance
        
        # Normaliser les poids
        if total_performance > 0:
            for name in weights:
                weights[name] /= total_performance
        
        return weights


# ============================================================
#         MODULE D'ANALYSE DES RÉPÉTITIONS
# ============================================================

class RepetitionAnalyzer:
    """
    Analyse les patterns de répétition entre tirages consécutifs.
    Identifie les numéros qui tendent à se répéter.
    """
    
    def __init__(self, config: GameConfig):
        self.config = config
        self.historique: List[List[int]] = []
        
        # Statistiques de répétition
        self.repetition_counts: Dict[int, int] = defaultdict(int)
        self.consecutive_counts: Dict[int, List[int]] = defaultdict(list)
        self.pair_repetitions: Counter = Counter()
        
    def analyze(self, historique: List[List[int]]):
        """Analyse complète des répétitions."""
        self.historique = historique
        self._analyze_single_repetitions()
        self._analyze_consecutive_runs()
        self._analyze_pair_repetitions()
        
    def _analyze_single_repetitions(self):
        """Analyse les répétitions simples (d'un tirage au suivant)."""
        self.repetition_counts = defaultdict(int)
        
        for i in range(1, len(self.historique)):
            prev_draw = set(self.historique[i-1])
            curr_draw = set(self.historique[i])
            
            repeated = prev_draw & curr_draw
            for num in repeated:
                self.repetition_counts[num] += 1
    
    def _analyze_consecutive_runs(self):
        """Analyse les séries consécutives de répétitions."""
        self.consecutive_counts = defaultdict(list)
        
        for num in range(1, self.config.num_max + 1):
            current_run = 0
            
            for i, draw in enumerate(self.historique):
                if num in draw:
                    current_run += 1
                else:
                    if current_run > 1:
                        self.consecutive_counts[num].append(current_run)
                    current_run = 0
            
            if current_run > 1:
                self.consecutive_counts[num].append(current_run)
    
    def _analyze_pair_repetitions(self):
        """Analyse les paires de numéros qui se répètent ensemble."""
        self.pair_repetitions = Counter()
        
        for i in range(1, len(self.historique)):
            prev_pairs = set(combinations(sorted(self.historique[i-1]), 2))
            curr_pairs = set(combinations(sorted(self.historique[i]), 2))
            
            repeated_pairs = prev_pairs & curr_pairs
            self.pair_repetitions.update(repeated_pairs)
    
    def get_repetition_probability(self) -> Dict[int, float]:
        """Calcule la probabilité de répétition pour chaque numéro."""
        total_opportunities = len(self.historique) - 1
        if total_opportunities <= 0:
            return {}
        
        probs = {}
        for num in range(1, self.config.num_max + 1):
            probs[num] = self.repetition_counts[num] / total_opportunities
        
        return probs
    
    def get_hot_repeaters(self, n: int = 10) -> List[Tuple[int, int]]:
        """Retourne les numéros qui se répètent le plus souvent."""
        sorted_nums = sorted(
            self.repetition_counts.items(),
            key=lambda x: x[1],
            reverse=True
        )
        return sorted_nums[:n]
    
    def get_current_candidates(self, last_draw: List[int]) -> Dict[int, float]:
        """
        Retourne les candidats à la répétition basés sur le dernier tirage.
        Score = probabilité historique de répétition.
        """
        rep_probs = self.get_repetition_probability()
        
        candidates = {}
        for num in last_draw:
            candidates[num] = rep_probs.get(num, 0)
        
        return candidates
    
    def get_repetition_stats(self) -> Dict:
        """Retourne les statistiques globales de répétition."""
        if len(self.historique) < 2:
            return {}
        
        # Compter les répétitions par tirage
        reps_per_draw = []
        for i in range(1, len(self.historique)):
            prev = set(self.historique[i-1])
            curr = set(self.historique[i])
            reps_per_draw.append(len(prev & curr))
        
        return {
            'avg_repetitions_per_draw': np.mean(reps_per_draw),
            'std_repetitions': np.std(reps_per_draw),
            'max_repetitions': max(reps_per_draw),
            'min_repetitions': min(reps_per_draw),
            'distribution': dict(Counter(reps_per_draw)),
            'zero_repetition_rate': reps_per_draw.count(0) / len(reps_per_draw)
        }
    
    def display_analysis(self):
        """Affiche l'analyse des répétitions."""
        print("\n" + "="*70)
        print(f"{'🔄 ANALYSE DES RÉPÉTITIONS':^70}")
        print("="*70)
        
        stats = self.get_repetition_stats()
        if not stats:
            print("Pas assez de données pour l'analyse.")
            return
        
        print(f"\n📊 STATISTIQUES GLOBALES:")
        print(f"   • Répétitions moyennes par tirage: {stats['avg_repetitions_per_draw']:.2f}")
        print(f"   • Écart-type: {stats['std_repetitions']:.2f}")
        print(f"   • Maximum observé: {stats['max_repetitions']}")
        print(f"   • Tirages sans répétition: {stats['zero_repetition_rate']*100:.1f}%")
        
        print(f"\n📈 DISTRIBUTION DES RÉPÉTITIONS:")
        for count, freq in sorted(stats['distribution'].items()):
            pct = freq / sum(stats['distribution'].values()) * 100
            bar = "█" * int(pct / 2)
            print(f"   {count} répétitions: {freq:4d} ({pct:5.1f}%) {bar}")
        
        print(f"\n🔥 TOP 10 NUMÉROS QUI SE RÉPÈTENT LE PLUS:")
        hot_repeaters = self.get_hot_repeaters(10)
        for num, count in hot_repeaters:
            prob = self.get_repetition_probability()[num]
            print(f"   N°{num:2d}: {count:3d} fois ({prob*100:.1f}%)")
        
        print(f"\n🤝 PAIRES QUI SE RÉPÈTENT LE PLUS:")
        top_pairs = self.pair_repetitions.most_common(5)
        for pair, count in top_pairs:
            print(f"   {pair}: {count} fois")


# ============================================================
#         MODULE DE RÉPÉTITION POUR IA LABO
# ============================================================

class PredictionModule(ABC):
    """Interface commune pour tous les modules."""
    
    def __init__(self, name: str, config: GameConfig):
        self.name = name
        self.config = config
        self.historique: List[List[int]] = []
        self.scores: Dict[int, float] = {}
        self.confidence: float = 1.0
        
    @abstractmethod
    def compute_scores(self) -> Dict[int, float]:
        pass
    
    def get_top_numbers(self, n: int) -> List[int]:
        sorted_nums = sorted(self.scores.items(), key=lambda x: x[1], reverse=True)
        return [num for num, score in sorted_nums[:n]]


class RepetitionModule(PredictionModule):
    """
    Module de prédiction basé sur l'analyse des répétitions.
    Combine probabilité de répétition et tendances récentes.
    """
    
    def __init__(self, config: GameConfig, 
                 repeat_weight: float = 0.6,
                 trend_weight: float = 0.4):
        super().__init__("Répétitions", config)
        self.repeat_weight = repeat_weight
        self.trend_weight = trend_weight
        self.analyzer = RepetitionAnalyzer(config)
        
    def compute_scores(self) -> Dict[int, float]:
        if len(self.historique) < 2:
            return {i: 0.0 for i in range(1, self.config.num_max + 1)}
        
        # Analyser les répétitions
        self.analyzer.analyze(self.historique)
        
        # Obtenir les probabilités de répétition
        rep_probs = self.analyzer.get_repetition_probability()
        
        # Obtenir les candidats du dernier tirage
        last_draw = self.historique[-1]
        candidates = self.analyzer.get_current_candidates(last_draw)
        
        # Calculer les scores
        for num in range(1, self.config.num_max + 1):
            rep_score = rep_probs.get(num, 0)
            
            # Bonus si le numéro est dans le dernier tirage
            if num in candidates:
                candidate_score = candidates[num]
            else:
                candidate_score = 0
            
            # Score combiné
            self.scores[num] = (
                self.repeat_weight * candidate_score + 
                self.trend_weight * rep_score
            )
        
        # Normaliser
        max_score = max(self.scores.values()) if max(self.scores.values()) > 0 else 1
        for num in self.scores:
            self.scores[num] /= max_score
        
        return self.scores


# ============================================================
#         MODULES EXISTANTS (avec améliorations)
# ============================================================

class FrequencyModule(PredictionModule):
    def __init__(self, config: GameConfig):
        super().__init__("Fréquence", config)
        
    def compute_scores(self) -> Dict[int, float]:
        if not self.historique:
            return {i: 0.0 for i in range(1, self.config.num_max + 1)}
        
        all_nums = [n for draw in self.historique for n in draw]
        freq = Counter(all_nums)
        
        max_freq = max(freq.values()) if freq else 1
        for i in range(1, self.config.num_max + 1):
            self.scores[i] = freq.get(i, 0) / max_freq
            
        return self.scores


class GapModule(PredictionModule):
    def __init__(self, config: GameConfig):
        super().__init__("Écarts", config)
        
    def compute_scores(self) -> Dict[int, float]:
        if not self.historique:
            return {i: 0.0 for i in range(1, self.config.num_max + 1)}
        
        last_seen = {i: -1 for i in range(1, self.config.num_max + 1)}
        
        for idx, draw in enumerate(self.historique):
            for num in draw:
                last_seen[num] = idx
        
        last_idx = len(self.historique) - 1
        gaps = {}
        for num, seen_idx in last_seen.items():
            gaps[num] = (last_idx - seen_idx) if seen_idx != -1 else len(self.historique)
        
        max_gap = max(gaps.values()) if gaps else 1
        for num, gap in gaps.items():
            self.scores[num] = gap / max_gap
            
        return self.scores


class MarkovModule(PredictionModule):
    def __init__(self, config: GameConfig):
        super().__init__("Markov", config)
        
    def compute_scores(self) -> Dict[int, float]:
        if len(self.historique) < 2:
            return {i: 0.0 for i in range(1, self.config.num_max + 1)}
        
        transitions = Counter()
        for i in range(len(self.historique) - 1):
            for num in self.historique[i]:
                for next_num in self.historique[i + 1]:
                    transitions[(num, next_num)] += 1
        
        last_draw = self.historique[-1]
        transition_scores = Counter()
        
        for num in last_draw:
            for (from_num, to_num), count in transitions.items():
                if from_num == num:
                    transition_scores[to_num] += count
        
        max_score = max(transition_scores.values()) if transition_scores else 1
        for i in range(1, self.config.num_max + 1):
            self.scores[i] = transition_scores.get(i, 0) / max_score
            
        return self.scores


# ============================================================
#         RÉSEAU DE NEURONES AMÉLIORÉ
# ============================================================

class NeuralNetworkSGD:
    """Réseau de neurones avec SGD."""
    
    def __init__(self, input_size: int = 70, hidden_layers: List[int] = [128, 64],
                 output_size: int = 70, learning_rate: float = 0.001):
        self.input_size = input_size
        self.hidden_layers = hidden_layers
        self.output_size = output_size
        self.learning_rate = learning_rate
        
        self.weights = []
        self.biases = []
        
        layer_sizes = [input_size] + hidden_layers + [output_size]
        
        for i in range(len(layer_sizes) - 1):
            limit = np.sqrt(6 / (layer_sizes[i] + layer_sizes[i+1]))
            w = np.random.uniform(-limit, limit, (layer_sizes[i], layer_sizes[i+1]))
            b = np.zeros((1, layer_sizes[i+1]))
            self.weights.append(w)
            self.biases.append(b)
        
        self.activations = []
        
    def forward(self, X: np.ndarray) -> np.ndarray:
        self.activations = [X]
        current = X
        
        for i, (w, b) in enumerate(zip(self.weights, self.biases)):
            z = np.dot(current, w) + b
            current = sigmoid(z)
            self.activations.append(current)
        
        return current
    
    def backward(self, y_true: np.ndarray, epsilon: float = 1e-7):
        m = y_true.shape[0] if len(y_true.shape) > 1 else 1
        y_pred = self.activations[-1]
        
        delta = (-y_true / (y_pred + epsilon) + (1 - y_true) / (1 - y_pred + epsilon))
        delta = delta * sigmoid_derivative(y_pred)
        
        deltas = [delta]
        
        for i in range(len(self.weights) - 2, -1, -1):
            delta = np.dot(deltas[-1], self.weights[i + 1].T)
            delta = delta * sigmoid_derivative(self.activations[i + 1])
            deltas.append(delta)
        
        deltas.reverse()
        
        for i in range(len(self.weights)):
            activation = self.activations[i]
            if len(activation.shape) == 1:
                activation = activation.reshape(1, -1)
            if len(deltas[i].shape) == 1:
                deltas[i] = deltas[i].reshape(1, -1)
                
            self.weights[i] -= self.learning_rate * np.dot(activation.T, deltas[i]) / m
            self.biases[i] -= self.learning_rate * np.mean(deltas[i], axis=0, keepdims=True)
    
    def train(self, X: np.ndarray, y: np.ndarray, epochs: int = 100) -> List[float]:
        losses = []
        for epoch in range(epochs):
            y_pred = self.forward(X)
            y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
            loss = -np.mean(y * np.log(y_pred_clipped) + (1 - y) * np.log(1 - y_pred_clipped))
            losses.append(loss)
            self.backward(y)
        return losses


class NeuralNetworkModule(PredictionModule):
    """Module réseau de neurones avec features améliorées."""
    
    def __init__(self, config: GameConfig, hidden_layers: List[int] = [128, 64],
                 epochs: int = 50, learning_rate: float = 0.001):
        super().__init__("NeuralNet", config)
        self.hidden_layers = hidden_layers
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.network = None
        self.training_losses = []
        
    def _compute_features(self) -> Tuple[np.ndarray, np.ndarray]:
        n = len(self.historique)
        X = np.zeros((n, self.config.num_max))
        y = np.zeros((n, self.config.num_max))
        
        freq_cumul = np.zeros(self.config.num_max)
        dejavu = np.zeros(self.config.num_max)
        
        for i, draw in enumerate(self.historique):
            # Mise à jour déjà vu
            dejavu += 1
            for num in draw:
                if 1 <= num <= self.config.num_max:
                    dejavu[num - 1] = 0
                    freq_cumul[num - 1] += 1
            
            # Features
            total_freq = np.sum(freq_cumul) if np.sum(freq_cumul) > 0 else 1
            X[i] = 0.5 * (freq_cumul / total_freq) + 0.5 * (dejavu / (dejavu.max() + 1))
            
            # Cibles
            for num in draw:
                if 1 <= num <= self.config.num_max:
                    y[i, num - 1] = 1
        
        return np.clip(X, 0, 0.99), np.clip(y, 0, 0.99)
        
    def compute_scores(self) -> Dict[int, float]:
        if len(self.historique) < 10:
            return {i: 0.5 for i in range(1, self.config.num_max + 1)}
        
        X, y = self._compute_features()
        
        self.network = NeuralNetworkSGD(
            input_size=self.config.num_max,
            hidden_layers=self.hidden_layers,
            output_size=self.config.num_max,
            learning_rate=self.learning_rate
        )
        
        self.training_losses = self.network.train(X, y, epochs=self.epochs)
        
        predictions = self.network.forward(X[-1:, :])
        output = predictions.flatten()
        
        for i in range(self.config.num_max):
            self.scores[i + 1] = float(output[i])
        
        return self.scores


# ============================================================
#              IA LABO AVEC APPRENTISSAGE ADAPTATIF
# ============================================================

class IALaboAdaptive:
    """
    IA Labo avec système d'apprentissage adaptatif.
    - Analyse les performances passées
    - Ajuste les poids automatiquement
    - Intègre l'analyse des répétitions
    """
    
    def __init__(self, config: GameConfig):
        self.config = config
        self.historique: List[List[int]] = []
        
        # Modules de prédiction
        self.modules: List[PredictionModule] = [
            FrequencyModule(config),
            MarkovModule(config),
            GapModule(config),
            RepetitionModule(config),  # Nouveau module
            NeuralNetworkModule(config, epochs=50),
        ]
        
        # Systèmes d'analyse
        self.performance_tracker = PerformanceTracker(config)
        self.repetition_analyzer = RepetitionAnalyzer(config)
        self.backtest_engine = None
        
        # Poids adaptatifs
        self.weights: Dict[str, float] = {m.name: 1.0 for m in self.modules}
        self.weight_history: List[Dict[str, float]] = []
        
        self._load_data()
        
    def _load_data(self):
        """Charge les données."""
        try:
            df = pd.read_csv(self.config.filepath, sep=';')
            num_cols = [col for col in df.columns if col.startswith('num')][:self.config.nums_per_draw]
            if not num_cols:
                num_cols = df.select_dtypes(include=[np.number]).columns[:self.config.nums_per_draw].tolist()
            
            self.historique = df[num_cols].dropna().apply(
                lambda x: sorted([int(v) for v in x.tolist()]), axis=1
            ).tolist()
            
            for module in self.modules:
                module.historique = self.historique
                
            self.backtest_engine = BacktestEngine(self.config, self.historique)
            self.repetition_analyzer.analyze(self.historique)
            
            print(f"✅ {len(self.historique)} tirages chargés")
            
        except FileNotFoundError:
            print(f"⚠️  Mode démo activé")
            import random
            self.historique = [
                sorted(random.sample(range(1, self.config.num_max + 1), self.config.nums_per_draw))
                for _ in range(500)
            ]
            for module in self.modules:
                module.historique = self.historique
            self.backtest_engine = BacktestEngine(self.config, self.historique)
            self.repetition_analyzer.analyze(self.historique)
    
    def run_full_backtest(self, train_size: int = 100, test_size: int = 50):
        """Exécute un backtest complet et ajuste les poids."""
        print("\n" + "="*70)
        print(f"{'📊 BACKTEST ET CALIBRATION DES POIDS':^70}")
        print("="*70)
        
        results_df = self.backtest_engine.compare_modules(
            self.modules,
            train_size=train_size,
            test_size=test_size
        )
        
        if results_df.empty:
            print("Pas de résultats de backtest.")
            return
        
        print("\n📈 RÉSULTATS DU BACKTEST:")
        print(results_df[['module_name', 'avg_hits', 'std_hits', 'max_hits', 'avg_hit_rate']].to_string())
        
        # Calculer les nouveaux poids
        optimal_weights = self.backtest_engine.get_optimal_weights()
        
        print("\n⚖️  NOUVEAUX POIDS CALCULÉS:")
        for name, weight in sorted(optimal_weights.items(), key=lambda x: x[1], reverse=True):
            old_weight = self.weights.get(name, 1.0)
            change = ((weight / (old_weight / len(self.modules))) - 1) * 100
            arrow = "↑" if change > 0 else "↓" if change < 0 else "→"
            print(f"   {name:15}: {weight:.3f} {arrow} ({change:+.1f}%)")
        
        # Sauvegarder l'historique des poids
        self.weight_history.append(self.weights.copy())
        self.weights.update(optimal_weights)
        
        return results_df
    
    def analyze_prediction_accuracy(self):
        """Analyse la précision des prédictions par numéro."""
        accuracy = self.performance_tracker.get_number_accuracy()
        
        if not accuracy or all(v == 0 for v in accuracy.values()):
            print("Pas encore de données de prédiction enregistrées.")
            return
        
        print("\n" + "="*70)
        print(f"{'🎯 PRÉCISION PAR NUMÉRO':^70}")
        print("="*70)
        
        sorted_accuracy = sorted(accuracy.items(), key=lambda x: x[1], reverse=True)
        
        print("\n   TOP 10 NUMÉROS LES PLUS PRÉCIS:")
        for num, acc in sorted_accuracy[:10]:
            bar = "█" * int(acc * 30)
            print(f"   N°{num:2d}: {acc*100:5.1f}% {bar}")
        
        print("\n   10 NUMÉROS LES MOINS PRÉCIS:")
        for num, acc in sorted_accuracy[-10:]:
            print(f"   N°{num:2d}: {acc*100:5.1f}%")
    
    def _compute_all_scores(self):
        """Calcule les scores de tous les modules."""
        print("\n⚙️  Calcul des scores...")
        for module in self.modules:
            module.historique = self.historique
            module.compute_scores()
            print(f"   ✓ {module.name}")
    
    def aggregate_with_adaptive_weights(self) -> Dict[int, float]:
        """Agrégation avec poids adaptatifs basés sur le backtest."""
        aggregated = {i: 0.0 for i in range(1, self.config.num_max + 1)}
        total_weight = sum(self.weights.get(m.name, 1.0) for m in self.modules)
        
        for num in aggregated:
            weighted_sum = sum(
                module.scores.get(num, 0) * self.weights.get(module.name, 1.0)
                for module in self.modules
            )
            aggregated[num] = weighted_sum / total_weight
        
        return aggregated
    
    def generate_predictions(self, n: int = 5) -> List[List[int]]:
        """Génère des prédictions avec le système adaptatif."""
        self._compute_all_scores()
        final_scores = self.aggregate_with_adaptive_weights()
        
        # Intégrer les informations de répétition
        rep_candidates = self.repetition_analyzer.get_current_candidates(self.historique[-1])
        rep_stats = self.repetition_analyzer.get_repetition_stats()
        expected_reps = rep_stats.get('avg_repetitions_per_draw', 0)
        
        sorted_nums = sorted(final_scores.items(), key=lambda x: x[1], reverse=True)
        top_pool = [num for num, _ in sorted_nums[:35]]
        
        combinations_list = []
        
        for i in range(n):
            if i == 0:
                # Première grille: top scores avec répétitions probables
                combo = []
                
                # Ajouter quelques répétitions probables
                rep_nums = sorted(rep_candidates.items(), key=lambda x: x[1], reverse=True)
                reps_to_add = min(int(expected_reps + 0.5), len(rep_nums))
                
                for num, _ in rep_nums[:reps_to_add]:
                    if num not in combo:
                        combo.append(num)
                
                # Compléter avec les meilleurs scores
                for num, _ in sorted_nums:
                    if len(combo) >= self.config.nums_per_draw:
                        break
                    if num not in combo:
                        combo.append(num)
                
                combo = sorted(combo[:self.config.nums_per_draw])
            else:
                # Autres grilles: échantillonnage pondéré
                weights = [final_scores[num] for num in top_pool]
                total_w = sum(weights)
                probs = [w/total_w for w in weights]
                
                combo = list(np.random.choice(
                    top_pool,
                    size=self.config.nums_per_draw,
                    replace=False,
                    p=probs
                ))
                combo = sorted(combo)
            
            combinations_list.append(combo)
        
        return combinations_list
    
    def record_actual_result(self, actual_numbers: List[int]):
        """
        Enregistre un résultat réel pour améliorer les prédictions futures.
        Doit être appelé après chaque tirage avec les numéros gagnants.
        """
        # Ajouter à l'historique
        self.historique.append(sorted(actual_numbers))
        
        # Mettre à jour les modules
        for module in self.modules:
            module.historique = self.historique
        
        # Mettre à jour l'analyseur de répétitions
        self.repetition_analyzer.analyze(self.historique)
        
        print(f"✅ Résultat enregistré: {actual_numbers}")
        print(f"   Historique: {len(self.historique)} tirages")
    
    def display_full_analysis(self):
        """Affiche l'analyse complète."""
        print("\n" + "="*70)
        print(f"{'🔬 IA LABO ADAPTATIF - ANALYSE COMPLÈTE':^70}")
        print(f"{'Jeu: ' + self.config.name:^70}")
        print("="*70)
        
        # Backtest et calibration
        self.run_full_backtest()
        
        # Analyse des répétitions
        self.repetition_analyzer.display_analysis()
        
        # Scores par module
        self._compute_all_scores()
        
        print("\n" + "="*70)
        print(f"{'📊 TOP 10 PAR MODULE':^70}")
        print("-"*70)
        for module in self.modules:
            top10 = module.get_top_numbers(10)
            weight = self.weights.get(module.name, 1.0)
            print(f"   {module.name:15} (poids: {weight:.2f}) → {top10}")
        
        # Prédictions finales
        print("\n" + "="*70)
        print(f"{'🎰 PRÉDICTIONS ADAPTATIVES':^70}")
        print("-"*70)
        
        predictions = self.generate_predictions(5)
        
        # Info sur les répétitions attendues
        rep_stats = self.repetition_analyzer.get_repetition_stats()
        print(f"\n   💡 Répétitions attendues: ~{rep_stats.get('avg_repetitions_per_draw', 0):.1f} par tirage")
        
        last_draw = self.historique[-1]
        print(f"   📌 Dernier tirage: {last_draw}")
        
        print("\n   GRILLES SUGGÉRÉES:")
        for i, combo in enumerate(predictions, 1):
            # Identifier les répétitions
            reps = set(combo) & set(last_draw)
            rep_info = f" (rép: {list(reps)})" if reps else ""
            print(f"      Grille {i}: {combo} | Somme: {sum(combo):3d}{rep_info}")
        
        # Statistiques de performance
        perf = self.performance_tracker.get_overall_performance()
        if perf:
            print("\n" + "="*70)
            print(f"{'📈 HISTORIQUE DE PERFORMANCE':^70}")
            print("-"*70)
            print(f"   Total prédictions enregistrées: {perf.get('total_records', 0)}")
            print(f"   Hits moyens: {perf.get('avg_hits', 0):.2f}")
            print(f"   Meilleur score: {perf.get('best_ever', 0)}")
        
        print("\n" + "="*70)
        print("="*70)


# ============================================================
#                      EXÉCUTION
# ============================================================

if __name__ == "__main__":
    print("""
    ╔══════════════════════════════════════════════════════════════════╗
    ║                                                                  ║
    ║        🧠 IA LABO ADAPTATIF - ANALYSE AVANCÉE 🧠                ║
    ║     Backtesting + Répétitions + Apprentissage Adaptatif         ║
    ║                                                                  ║
    ╚══════════════════════════════════════════════════════════════════╝
    """)
    
    # Configuration
    config = CONFIGS["loto"]  # Changer selon le jeu
    
    try:
        # Initialiser IA Labo Adaptatif
        lab = IALaboAdaptive(config)
        
        # Afficher l'analyse complète
        lab.display_full_analysis()
        
        # Exemple: enregistrer un nouveau résultat
        # lab.record_actual_result([3, 15, 22, 38, 45])
        
        # Exemple: sauvegarder l'historique
        # lab.performance_tracker.save_to_file()
        
    except Exception as e:
        print(f"❌ Erreur: {e}")
        import traceback
        traceback.print_exc()


    ╔══════════════════════════════════════════════════════════════════╗
    ║                                                                  ║
    ║        🧠 IA LABO ADAPTATIF - ANALYSE AVANCÉE 🧠                ║
    ║     Backtesting + Répétitions + Apprentissage Adaptatif         ║
    ║                                                                  ║
    ╚══════════════════════════════════════════════════════════════════╝
    
✅ 960 tirages chargés

                🔬 IA LABO ADAPTATIF - ANALYSE COMPLÈTE                
                              Jeu: Loto                               

                 📊 BACKTEST ET CALIBRATION DES POIDS                  

🔬 Backtesting en cours...
   Testing Fréquence... ✓ (avg hits: 0.48)
   Testing Markov... ✓ (avg hits: 0.47)
   Testing Écarts... ✓ (avg hits: 0.53)
   Testing Répétitions... ✓ (avg hits: 0.52)
   Testing NeuralNet... ✓ (avg hits: 0.53)

📈 RÉSULTATS DU BACKTEST:
   module_name  avg_hits  std_hits  max_hits  avg_hit_rate
